In [1]:
import numpy as np
import csv
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import tensorflow.keras.datasets.mnist as mnist

In [2]:
(xtrain_raw,ytrain_raw),(xtest_raw,ytest_raw) = mnist.load_data()

training_index_3 = ytrain_raw ==3
training_index_4 = ytrain_raw ==4
training_index = training_index_3 | training_index_4

test_index_3 = ytest_raw ==3
test_index_4 = ytest_raw ==4
test_index = test_index_3 | test_index_4

xtrain = xtrain_raw[training_index]
ytrain = ytrain_raw[training_index]
xtest = xtest_raw[test_index]
ytest = ytest_raw[test_index]
xtrain = np.reshape(xtrain,(len(xtrain),784))
xtest  = np.reshape(xtest ,(len(xtest ),784))

pca = PCA(n_components=1)

xtrain = pca.fit_transform(xtrain)
xtest  = pca.transform(xtest)

In [3]:

## scale training set to [0,1]

vmax = max(np.max(xtrain),np.max(xtest))
vmin = min(np.min(xtrain),np.min(xtest))
xtrain = (xtrain-vmin)/(vmax-vmin)
xtest = (xtest-vmin)/(vmax-vmin)


In [4]:

xtrain3 = xtrain[ytrain ==3]
xtrain4 = xtrain[ytrain ==4] 

In [5]:
mean_3 = np.mean(xtrain3)
mean_4 = np.mean(xtrain4)
lam_3 = 1/mean_3
lam_4 = 1/mean_4

In [6]:
cov_3 = np.mean(xtrain3**2)
cov_4 = np.mean(xtrain4**2)

In [7]:
prob_3 = (1/(cov_3*2*np.pi))**0.5*np.exp(-0.5*(xtest-mean_3)**2/cov_3)
prob_4 = (1/(cov_4*2*np.pi))**0.5*np.exp(-0.5*(xtest-mean_4)**2/cov_4)

In [8]:
predict = np.zeros(len(ytest))
predict[prob_3[:,0]>=prob_4[:,0]] = 3
predict[prob_3[:,0]<prob_4[:,0]] = 4

In [9]:
acc_med = sum(predict==ytest)/len(ytest)
print("The accuracy of 2 parameter model is %.3f%%"%(acc_med*100))

The accuracy of 2 parameter model is 87.651%


In [10]:
prob_3 = 0.5*(1/(cov_3*2*np.pi))**0.5*np.exp(-0.5*(xtest-mean_3)**2/cov_3)*lam_3*np.exp(-lam_3*xtest)
prob_4 = 0.5*(1/(cov_4*2*np.pi))**0.5*np.exp(-0.5*(xtest-mean_4)**2/cov_4)*lam_4*np.exp(-lam_4*xtest)
predict = np.zeros(len(ytest))
predict[prob_3[:,0]>=prob_4[:,0]] = 3
predict[prob_3[:,0]<prob_4[:,0]] = 4
acc_med = sum(predict==ytest)/len(ytest)
print("The accuracy of 3 parameter model is %.3f%%"%(acc_med*100))

The accuracy of 3 parameter model is 94.378%


In [11]:
ss = 20
prob_3 = np.mean(1/(ss*2*np.pi)**0.5*np.exp(-(xtrain3-xtest.T)**2/ss),axis = 0)
prob_4 = np.mean(1/(ss*2*np.pi)**0.5*np.exp(-(xtrain4-xtest.T)**2/ss),axis = 0)
predict = np.zeros(len(ytest))
predict[prob_3>=prob_4] = 3
predict[prob_3<prob_4] = 4
acc_med = sum(predict==ytest)/len(ytest)
print("The accuracy of kernel based model is %.3f%%"%(acc_med*100))

The accuracy of kernel based model is 97.741%


In [12]:
np.var(xtest)

0.06153576617373428